**<font size = 5 > 0. SETUP LIBRARIES AND PACKAGES** </font>

In [1]:
#####--- 0.1 Import libraries
import tweepy
from kafka import KafkaProducer
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from tweepy import Stream
import json
from pathlib import Path
from ruamel.yaml import YAML
import pandas as pd
from  datetime import datetime
import time
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')


#####--- 0.1 Define class used later in the program
class MyListener(StreamListener):
    def __init__(self, producer, name_pgr = ""):
        super().__init__()
        self.producer = producer
        self.pgr = name_pgr
    
    def on_status(self, status):
        tweet = {
            
          #In this section we get users' attribute
            
          'user_id': status.user.id,
          'username': status.user.name,
          'created_at': str(status.created_at),
          'followers':status.user.followers_count,
          'friends':status.user.friends_count,
          'location': status.user.location,
            
          #In this section we get others attributes
            
          'text': status.text,
          'hashtags': [],
          'user_mentions_id': [],
          'user_mentions_name': [] ,
            
            
          #In this section we add some attributes from external file
            
          'nome_pgr':self.pgr
        }
        if status.entities.get('hashtags') is not None:
            hashtags = status.entities.get('hashtags')
            tweet['hashtags'] = [ h.get('text') for h in hashtags ]
        if status.entities.get('user_mentions') is not None:
            mentions = status.entities.get('user_mentions')
            tweet['user_mentions_id'] = [ m.get('id') for m in mentions ]
            tweet['user_mentions_name'] = [ m.get('name') for m in mentions ]
            
    
        self.producer.send(topic = act_topic, value = tweet)



**<font size = 5 > 1. IMPORT DATASETS AND CONFIG_FILE** </font>

In [2]:
#####--- 1.0 Read Config_File and set some variables
yaml        = YAML(typ='safe')
with open('Config_File.yml', 'r') as fil:
        config = yaml.load(fil)
config_general = config["general"]
config_pgr = config["pgr1"]
nome_pgr = config_pgr["programma"].capitalize()
act_topic = config_pgr["topic"]
hours_gtm = int(config_general["hours_gtm"])
delta_before = int(config_pgr["delta_before"])
leng = config_pgr["leng"]

#####--- 1.1 Read third part data and set some variables
df_pgr = pd.read_excel(config_general["pgr_data"])
df_subset = df_pgr.loc[df_pgr["Nome programma"] == nome_pgr]
programma = df_subset["Nome programma"].values[0]
h_ini = df_subset["Orario inizio"].values[0]
h_ini = datetime.strptime(h_ini, "%Y-%m-%d %H:%M:%S")
keywords = df_subset["Hashtags"].values[0]

#####--- 1.2 Set consumer key, consumer secret, access token and access token secret
consumer_key = config_general["ck"]
consumer_secret = config_general["cs"]
access_token = config_general["at"]
access_token_secret = config_general["ats"]

#####--- 1.3 Authenticate with the OAuth protocol
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

#####--- 1.4 Get the API object passing the authentication object
api = tweepy.API(auth)

#####--- 1.5 Set producer
producer = KafkaProducer(
  bootstrap_servers=["kafka:9092"],
  value_serializer=lambda v: json.dumps(v).encode("utf-8"))

**<font size = 5 > 2. STREAM DATA** </font>

In [ ]:
#####--- 2.0 Define listener
listener = MyListener(producer = producer, name_pgr = programma)
#####--- 2.1 Define streamer
stream = Stream(auth = api.auth, listener=listener)

#####--- 2.2 Set program hour
ora_now = datetime.now()
ora_pgr = datetime(ora_now.year, ora_now.month, ora_now.day, 
                   h_ini.hour - hours_gtm, h_ini.minute, h_ini.second) 
inizio_stream = ora_pgr - timedelta(minutes = delta_before)
print("Orario adesso: ", ora_now)
print("Orario programma: ", ora_pgr)
print("Orario inizio streaam: ", inizio_stream)

#####--- 2.3 Wait until program start
while (datetime.now() < inizio_stream):
    time.sleep(5)
    print("\n waiting..\n")

#####--- 2.4 Start streaming
print("Stopped waiting..\n")
print("Start streaming..\n")
stream.filter(track = keywords, languages=[leng])